In [3]:

import numpy as np
import itertools
import random
from statistics import stdev
import math
import ast

In [4]:

p_running = 0.9
p_failure = 1 - p_running
path = '/home/dkabe/Model_brainstorming/SAA_Analysis_v7/Instance_1/'

In [5]:
N = [64, 128, 192, 256, 320, 384]
batches = 30
Products = 3
Market = 29
text_file = open(path + 'Disruption_scenarios.txt', "r")
ls = text_file.read().split('\n')[:-1]
Scenarios = list(map(lambda x: ast.literal_eval(x), ls))
Probabilities = np.loadtxt(path + "disruption_probabilities.txt")
num_disruption = len(Scenarios)

In [11]:
(Scenarios)

[[[1, 1, 1, 1, 1, 1], (1, 1, 1, 1)],
 [[1, 1, 1, 1, 1, 1], (1, 1, 1, 0)],
 [[1, 1, 1, 1, 1, 1], (1, 1, 0, 1)],
 [[1, 1, 1, 1, 1, 1], (1, 1, 0, 0)],
 [[1, 1, 1, 1, 1, 1], (1, 0, 1, 1)],
 [[1, 1, 1, 1, 1, 1], (1, 0, 1, 0)],
 [[1, 1, 1, 1, 1, 1], (1, 0, 0, 1)],
 [[1, 1, 1, 1, 1, 1], (1, 0, 0, 0)],
 [[1, 1, 1, 1, 1, 1], (0, 1, 1, 1)],
 [[1, 1, 1, 1, 1, 1], (0, 1, 1, 0)],
 [[1, 1, 1, 1, 1, 1], (0, 1, 0, 1)],
 [[1, 1, 1, 1, 1, 1], (0, 1, 0, 0)],
 [[1, 1, 1, 1, 1, 1], (0, 0, 1, 1)],
 [[1, 1, 1, 1, 1, 1], (0, 0, 1, 0)],
 [[1, 1, 1, 1, 1, 1], (0, 0, 0, 1)],
 [[1, 1, 1, 1, 1, 1], (0, 0, 0, 0)],
 [[1, 1, 1, 1, 0, 0], (1, 1, 1, 1)],
 [[1, 1, 1, 1, 0, 0], (1, 1, 1, 0)],
 [[1, 1, 1, 1, 0, 0], (1, 1, 0, 1)],
 [[1, 1, 1, 1, 0, 0], (1, 1, 0, 0)],
 [[1, 1, 1, 1, 0, 0], (1, 0, 1, 1)],
 [[1, 1, 1, 1, 0, 0], (1, 0, 1, 0)],
 [[1, 1, 1, 1, 0, 0], (1, 0, 0, 1)],
 [[1, 1, 1, 1, 0, 0], (1, 0, 0, 0)],
 [[1, 1, 1, 1, 0, 0], (0, 1, 1, 1)],
 [[1, 1, 1, 1, 0, 0], (0, 1, 1, 0)],
 [[1, 1, 1, 1, 0, 0], (0, 1, 0, 1)],
 

In [4]:
p1_mean = 103500
p2_mean = 101500
p3_mean = 99750

p1_sd = 1650
p2_sd = 4125
p3_sd = 2000

distribution_mat = [[p1_mean, p1_sd],
                  [p2_mean, p2_sd],
                  [p3_mean, p3_sd]]

In [5]:

np.random.seed(1)
nominal_demand = np.zeros((Products, Market))
for m in range(Products):
    nominal_demand[m] = np.round(np.random.normal(distribution_mat[0][0], distribution_mat[0][1], size = 29))
    nominal_demand[m] = np.round(np.random.normal(distribution_mat[1][0], distribution_mat[1][1], size = 29))
    nominal_demand[m] = np.round(np.random.normal(distribution_mat[2][0], distribution_mat[2][1], size = 29))

In [6]:
delta = np.arange(0.75, 1.255, 0.005)

In [7]:
delta

array([0.75 , 0.755, 0.76 , 0.765, 0.77 , 0.775, 0.78 , 0.785, 0.79 ,
       0.795, 0.8  , 0.805, 0.81 , 0.815, 0.82 , 0.825, 0.83 , 0.835,
       0.84 , 0.845, 0.85 , 0.855, 0.86 , 0.865, 0.87 , 0.875, 0.88 ,
       0.885, 0.89 , 0.895, 0.9  , 0.905, 0.91 , 0.915, 0.92 , 0.925,
       0.93 , 0.935, 0.94 , 0.945, 0.95 , 0.955, 0.96 , 0.965, 0.97 ,
       0.975, 0.98 , 0.985, 0.99 , 0.995, 1.   , 1.005, 1.01 , 1.015,
       1.02 , 1.025, 1.03 , 1.035, 1.04 , 1.045, 1.05 , 1.055, 1.06 ,
       1.065, 1.07 , 1.075, 1.08 , 1.085, 1.09 , 1.095, 1.1  , 1.105,
       1.11 , 1.115, 1.12 , 1.125, 1.13 , 1.135, 1.14 , 1.145, 1.15 ,
       1.155, 1.16 , 1.165, 1.17 , 1.175, 1.18 , 1.185, 1.19 , 1.195,
       1.2  , 1.205, 1.21 , 1.215, 1.22 , 1.225, 1.23 , 1.235, 1.24 ,
       1.245, 1.25 ])

In [8]:
demand_realization = len(delta)

In [9]:
np.mean(nominal_demand)

99878.62068965517

In [10]:
random.seed(1)
demand = np.zeros((demand_realization, Products, Market))
for r in range(demand_realization):
    demand[r] = delta[r]*nominal_demand

In [11]:
S = demand_realization*Scenarios
S_demand = []
S_prob = list(Probabilities)*demand_realization
multiplier = sum(S_prob)
S_prob = list(map(lambda x: x/multiplier, S_prob))
for r in range(demand_realization):
    S_demand+=([demand[r]]*num_disruption)

In [14]:
np.sum(S_prob)

0.9999999999998642

In [16]:
np.random.seed(2)
evaluation_indices = np.random.choice(range(len(S)), 1000, p = S_prob)
evaluation_set = [S[index] for index in evaluation_indices]
evaluation_demand = np.array([S_demand[index] for index in evaluation_indices])
eval_probabilities = [S_prob[index] for index in evaluation_indices]
#with open(path + 'Evaluation_Set.txt', 'w') as f:
 #   for item in evaluation_set:
  #      f.write("%s\n" % item)
#f.close()
#np.savetxt(path + "eval_set_demand.txt", evaluation_demand.reshape((1000*3,29)))

In [ ]:
np.random.seed(2)
evaluation_indices = np.random.choice(range(len(Scenarios)), 1000, p = Probabilities)
evaluation_set = [Scenarios[index] for index in evaluation_indices]
demand_indices = np.random.choice(range(len(demand)), 1000, p = 1/len(demand))
evaluation_demand = np.array([demand[index] for index in demand_indices])
#with open(path + 'Evaluation_Set.txt', 'w') as f:
 #   for item in evaluation_set:
  #      f.write("%s\n" % item)
#f.close()
#np.savetxt(path + "eval_set_demand.txt", evaluation_demand.reshape((1000*3,29)))


In [ ]:
N = [64, 128, 192, 256, 320, 384]
np.random.seed(1)
for num_scen in N:
    for batch in range(batches):
        indices = np.random.choice(range(len(S)), num_scen, p = S_prob)
        sample_scenarios = [S[index] for index in indices]
        sample_demand = np.array([S_demand[index] for index in indices])
        #with open(path + 'Scenarios/' + str(num_scen) + '_' + str(batch) + '.txt', 'w+') as f:
         #   for item in sample_scenarios:
          #      f.write("%s\n" % item)
        #f.close()
        #np.savetxt(path + 'Scen_demand/' + 'demand_' + str(num_scen) + '_' + str(batch) + '.txt', sample_demand.reshape((num_scen*Products, Market)))

In [ ]:
N = [64, 128, 192, 256, 320, 384]
np.random.seed(1)
for num_scen in N:
    for batch in range(batches):
        indices = np.random.choice(range(len(Scenarios)), num_scen, p = Probabilities)
        sample_scenarios = [Scenarios[index] for index in indices]
        demand_indices = np.random.choice(range(len(demand)), num_scen, p = 1/len(demand))
        sample_demand = np.array([demand[index] for index in demand_indices])
        #with open(path + 'Scenarios/' + str(num_scen) + '_' + str(batch) + '.txt', 'w+') as f:
         #   for item in sample_scenarios:
          #      f.write("%s\n" % item)
        #f.close()
        #np.savetxt(path + 'Scen_demand/' + 'demand_' + str(num_scen) + '_' + str(batch) + '.txt', sample_demand.reshape((num_scen*Products, Market)))

In [66]:
batches = 30
path = '/home/dkabe/Model_brainstorming/SAA_Analysis_v7/Instance_1/'
N = [64, 128, 192, 256, 320, 384]
for scen in N:
    gaps = []
    ub_averages = []
    ub_max = []
    objvals_lb = np.loadtxt(path + "Objectives/" + str(scen) + '_results.txt')
    avg_lb = (np.mean(objvals_lb))
    std_lb = (stdev(objvals_lb))
    lbwidth = std_lb/math.sqrt(batches)*2.045
    lbmin = round(avg_lb - lbwidth,2)
    #print(avg_lb, lbmin)
    #print('ci on lower bound = [', round(avg_lb-lbwidth,2), ',', round(avg_lb+lbwidth,2), ']')
    for batch in range(batches):
        objvals_ub = np.loadtxt(path + "Upper_Bounds/" + str(scen) + "_scenarios/" + str(scen) + "_" + str(batch) + "_UB_results.txt")
        avg_ub = (np.mean(objvals_ub))
        std_ub = stdev(objvals_lb)
        ubwidth = std_ub/math.sqrt(1000)*1.96
        ubmax = round(avg_ub + ubwidth, 2)
        gap = round(100*((ubmax - lbmin)/lbmin),2)
        ub_averages.append(avg_ub)
        gaps.append(gap)
        ub_max.append(ubmax)
    min_gap = min(filter(lambda x: x >= 0, gaps))
    #min_gap = min(gaps)
    index = gaps.index(min_gap)
    #print(ub_averages[index], ub_max[index], min_gap)
    print(avg_lb, "\t", ub_averages[index], "\t", lbmin, "\t", ub_max[index], "\t", min_gap, "\t")

18172722.04566667 	 18092861.457599998 	 17795081.45 	 18155552.0 	 2.03 	
18150771.880999997 	 18092861.457599998 	 17912477.98 	 18132419.64 	 1.23 	
18580983.086000003 	 18388823.38891 	 18395294.24 	 18419648.82 	 0.13 	
18440944.79233333 	 18308636.52167 	 18285644.6 	 18334417.26 	 0.27 	
18421424.822666667 	 18483934.14432 	 18297993.54 	 18504424.46 	 1.13 	
18449658.89266666 	 18308636.52167 	 18309694.44 	 18331871.44 	 0.12 	


In [42]:
# save final number of scenarios to use 
num_scen = 192
indices = np.random.choice(range(len(Scenarios)), num_scen, p = Probabilities)
sample_scenarios = [Scenarios[index] for index in indices]
demand_indices = np.random.choice(range(len(demand)), num_scen, p = 1/len(demand))
sample_demand = np.array([demand[index] for index in demand_indices])
#with open("/home/dkabe/Model_brainstorming/Input_Data/Realistic/Instance_1/scen_1.txt", 'w+') as f:
 #   for item in sample_scenarios:
  #      f.write("%s\n" % item)
#f.close()
#np.savetxt("/home/dkabe/Model_brainstorming/Input_Data/Realistic/Instance_1/Demand_1.txt", sample_demand.reshape((192*Products, Market)))